In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake_news_tuned_model/transformers/default/1/fake_news_trained_model
/kaggle/input/datess/embedding.npy
/kaggle/input/fake-news-detection-datasets/News _dataset/True.csv
/kaggle/input/fake-news-detection-datasets/News _dataset/Fake.csv


#cheking versions of the packages, since i dont have local computation i used kaagle for computation.

In [4]:
import sklearn
sklearn.__version__

'1.2.2'

In [3]:
np.__version__

'1.26.4'

In [5]:
pd.__version__

'2.2.3'

In [6]:
t_news = pd.read_csv('/kaggle/input/fake-news-detection-datasets/News _dataset/True.csv')
f_news = pd.read_csv('/kaggle/input/fake-news-detection-datasets/News _dataset/Fake.csv')
t_news['label'] = 1
f_news['label'] = 0

In [7]:
f_news['text'][2200]

'Oh, the wiretapping scandal. Or it would be a scandal if it were true. Trump hasn t produced any hard evidence of being tapped anywhere, but Republicans have jumped on the series of tweets in which he accused President Obama of tapping him as though it s concrete fact. Enter Rep. Steve King, who licks Trump s boots. He appeared on MSNBC last night to discuss Trump and his wiretapping nonsense.And he flat-out admitted that the GOP doesn t have any facts proving that Trump was wiretapped. Literally. He said (with a smile, of all things): We actually don t have any facts to work with on this. What we got are opinions. We got opinions from the intelligence community, from Clapper, from Brennan and others. He went on to say that he gets to hear the opinions of the intelligence community, but said again that they don t have any facts. That does not, however, change his belief that both Michael Flynn and Trump were wiretapped. He seems to think that the evidence is there, the intelligence is

In [8]:
f_news['title'][2200]

' WATCH: GOP Rep. Steve King Just Dropped A BOMBSHELL Admission About Trump’s Wiretapping Claims'

In [9]:
len('Oh, the wiretapping scandal. Or it would be a scandal if it were true. Trump hasn t produced any hard evidence of being tapped anywhere, but Republicans have jumped on the series of tweets in which he accused President Obama of tapping him as though it s concrete fact. Enter Rep. Steve King, who licks Trump s boots. He appeared on MSNBC last night to discuss Trump and his wiretapping nonsense.And he flat-out admitted that the GOP doesn t have any facts proving that Trump was wiretapped. Literally. He said (with a smile, of all things): We actually don t have any facts to work with on this. What we got are opinions. We got opinions from the intelligence community, from Clapper, from Brennan and others. He went on to say that he gets to hear the opinions of the intelligence community, but said again that they don t have any facts. That does not, however, change his belief that both Michael Flynn and Trump were wiretapped. He seems to think that the evidence is there, the intelligence is there, he just can t see it because it s sensitive.Or something.How much longer is this charade about wiretaps going to go on before they re forced to admit that they either never happened, or happened with good reason?  Watch King s admission below:Unreal. Steve King tells @MSNBC he suspects Flynn, Trump were wiretapped THEN admits  we actually don t have any facts to work with on this  pic.twitter.com/ubNAjZYrzI  Kyle Griffin (@kylegriffin1) March 11, 2017Featured image via screen capture from embedded video')

1515

In [10]:
t_news['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [11]:
df = pd.concat([t_news,f_news],axis = 0)

In [13]:
df['label'].value_counts()

label
0    23481
1    21417
Name: count, dtype: int64

target class is balanced pretty much so just letting it be it....

checking how much text length is greater than 512. i thinking about using bert  or DistilBERT at first 


In [14]:
def length_cal(df):
    
    return len(df) > 512

In [15]:
df['leng'] = df['text'].apply(length_cal)

In [16]:
sum(df['leng'])

40186

In [17]:
def counts(df):
    return len(df)

In [19]:
df['counts'] = df['text'].apply(counts)

checking average word lenght in my dataset,to make the final decision

In [20]:
np.mean(df['counts'])

2469.1096930820972

since avg word length is 2400, we can use sentenceformer which is best compared to tf-idf and word2vec.

In [21]:
df.head()

,title,text,subject,date,label,leng,counts
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1,True,4659
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1,True,4077
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1,True,2789
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1,True,2461
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1,True,5204


In [22]:
#checking categories for the fake news
a = df['subject'].unique()
for i in a:
    s = df[(df['subject'] == i) & (df['label'] == 0)]
    count = s.shape[0]
    print(f'fake news for {i} is {count}')

fake news for politicsNews is 0
fake news for worldnews is 0
fake news for News is 9050
fake news for politics is 6841
fake news for Government News is 1570
fake news for left-news is 4459
fake news for US_News is 783
fake news for Middle-east is 778


In [23]:
df.shape

(44898, 7)

In [24]:
df = df[['title','text','label']]

In [25]:
df

,title,text,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1
...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,0


shuffling dataset to avoid over usage of particular class.

In [ ]:
from sklearn.utils import shuffle
df = df.sample(frac=1,random_state = 42).reset_index(drop=True)

In [ ]:
df.head(10)
#transformers==4.30.2

In [1]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 31.5 MB/s eta 0:00:00
  Attempting uninstall: 

In [8]:
pip show sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: sentence-transformers
Version: 4.1.0
Summary: Embeddings, Retrieval, and Reranking
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers, typing_extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install sentence-transformers==2.2.2 huggingface-hub==0.15.1 accelerate==0.20.3

In [2]:
from sentence_transformers import SentenceTransformer
model = 'sentence-transformers/all-MiniLM-L6-v2'
sentence_model = SentenceTransformer(model)

2025-10-17 14:55:17.624685: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760712917.842747      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760712917.903173      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
preds = sentence_model.encode('i love python, python is good language  ')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
preds

In [ ]:
preds[0]

combining the title and text for content rich.

In [ ]:
combined_text = df['title'] + '.'+df['text']

In [ ]:
embedding = sentence_model.encode(combined_text)

In [ ]:
embedding

saving the trained embedding for future usage. i used light weighted sentence former for deployment,which have 384 dimention which is pretty good for my problem.

In [ ]:
np.save('embedding.npy',embedding)

In [30]:
embedding = np.load('/kaggle/input/datess/embedding.npy')

In [ ]:
#checking the dimentions
df_emb = pd.DataFrame(embedding)
df_emb['label'] = df['label'].to_list()

In [ ]:
df_emb.head()

In [29]:
#specifying the target variable
y = df['label']

In [31]:
#splitting the dataset for training an testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(embedding,y,test_size = 0.2,stratify=y,random_state=42)

In [ ]:
# am using random forsest which is both perform well on this type of data.
# it is also light weight for deployment.
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100,random_state=42)
model.fit(x_train,y_train)

In [32]:
y_pred = model.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test,y_pred)
print(report)

we got 96 and 97 f1 score and all other metrices were also greater than 92 which is good,
now save the model.

In [ ]:
import joblib
joblib.dump(model,'fake_news_trained_model')

In [7]:
!python --version

Python 3.11.13


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [28]:
import joblib
trained_model = joblib.load('/kaggle/input/fake_news_tuned_model/transformers/default/1/fake_news_trained_model')

In [5]:
trained_model.predict([preds])

array([1])